<a href="https://colab.research.google.com/github/faisalinayath/ASSIGNMENT/blob/main/Saved_Model_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers.optimization import  Adafactor
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json


--2023-10-15 18:30:41--  https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.51.200, 52.216.138.253, 52.217.32.158, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.51.200|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1199 (1.2K) [application/json]
Saving to: ‘t5-base-config.json’

t5-base-config.json 100%[===================>]   1.17K  --.-KB/s    in 0s      

2023-10-15 18:30:42 (82.6 MB/s) - ‘t5-base-config.json’ saved [1199/1199]



In [ ]:
from transformers import T5ForConditionalGeneration, T5Config

# Path to the uploaded model and configuration files
model_path = '/content/drive/MyDrive/Colab Notebooks/second_trained_model.bin'
config_path = 't5-base-config.json'

# Load the configuration
config = T5Config.from_json_file(config_path)

# Load the model
model_new = T5ForConditionalGeneration.from_pretrained(model_path, config=config)

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
import time
import torch

def generate(text, max_length=100):
    model_new.eval()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Choose the device
    model=model_new.to(device)
    input_ids = tokenizer.encode("WebNLG:{} ".format(text), return_tensors="pt").to(device)

    s = time.time()

    outputs = model.generate(input_ids, max_length=max_length)
    gen_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    elapsed = time.time() - s
    print('Generated in {} seconds'.format(str(elapsed)[:50]))
    return gen_text

# Example usage
generated_text = generate('Topic | Energy Management | Accounting Metric | (1) Total energy consumed, (2) percentage grid electricity, (3) percentage renewable | Category | Quantitative | Unit of Measure is Gigajoules (GJ), Percentage (%) | code is RT-IG-130a.1 | Disclosure is (1) 851,140 GJ (2) ~15% (3) ~16%')


Generated in 7.6871747970581055 seconds


In [ ]:
generated_text

"The RT-IG-130a.1 is the code for the RT-IG-130a. It's a total energy management system with a 2% energy consumed by a total energy consumed by a 2-part energy consumed by a Gigajoules (GJ), percentage (%) and a 5% (3) 15% (3) 15% (3) 15% (3) 16% (part of the category"

In [ ]:
generated_text=generate("Policy Number PLC011 | Seller: ABC Corporation | Buyer: XYZ Inc | Property: 456 Elm St Los Angeles | Type: Commercial | Policy Amount: $1200000 | Premium: $2500 | Agent: Laura Agent | Branch: Los Angeles Branch")
generated_text

Generated in 3.0856728553771973 seconds


"Policy PLC011 covers the entire length of 456 Elm St in Los Angeles. It's a $1200000 commercial property with a $2500 premium managed by Agent in the Los Angeles Branch."

In [ ]:
generated_text=generate("Transport Company: Swift Logistics | Shipment ID: TRS001 | Sender: ABC Manufacturing | Receiver: XYZ Retail | Shipment Details: 100 pallets of goods | Shipment Value: $50,000 | Carrier: Express Cargo | Route: New York to Los Angeles")
generated_text

Generated in 2.069889783859253 seconds


'The Carrier Express Cargo in New York to Los Angeles has a shipment ID of TRS001 and a $50,000 ship.'